In [1]:
import pandas as pd
import dask.dataframe as dd

df = pd.read_csv('./nocom_elo_scores.csv', dtype={'model_bad_id': 'object', 'model_good_id': 'object'})

In [2]:
df2 = pd.merge(df, df, left_on=['model_bad_id', 'model_good_id'], right_on=['model_good_id', 'model_bad_id'], suffixes=('_A', '_B'))
df2.drop(['model_bad_name_A', 'model_bad_step_A', 'model_good_name_A', 'model_good_step_A',  'model_bad_name_B', 'model_bad_step_B', 'model_good_name_B',
       'model_good_step_B'], axis=1, inplace=True)

df2.rename(columns={"model_bad_id_A": "model_id", "model_good_id_A": "adversary_id", "reward_bad_A": "bad_reward", "reward_good_B": "good_reward"}, inplace=True)
df2.drop(['reward_good_A', 'model_bad_id_B', 'model_good_id_B', 'reward_bad_B'], axis=1, inplace=True)
df2 = df2[df2['model_id'] != df2['adversary_id']]

In [3]:
def compute_offset(elos, a, b, scoreBad, scoreGood):
    eloA = elos[elos['model_id'] == a]['elo'].values[0]
    eloB = elos[elos['model_id'] == b]['elo'].values[0]
    S = (scoreBad + scoreGood) / 2
    E = 120 / (1 + 10 ** ((eloB - eloA) / 3500))
    return 25 * (S - E)

In [4]:
elos = pd.DataFrame({'model_id': df['model_bad_id'].unique(), 'elo': 1000})
for _ in range(20):
    elos_tmp = elos.copy()
    for i in range(len(elos)):
        name = elos['model_id'].iloc[i]
        offsets = df2[df2['model_id'] == name].apply(lambda x: compute_offset(elos, x['model_id'], x['adversary_id'], x['bad_reward'], x['good_reward']), axis=1)
        eloA = elos.iloc[i]['elo']
        elos_tmp.loc[i, 'elo'] = eloA + (offsets.sum() / len(offsets))
    elos = elos_tmp

In [5]:
elos_named = pd.merge(elos, df.drop(['model_good_id', 'model_good_name', 'model_good_step', 'reward_bad', 'reward_good'], axis=1).drop_duplicates(), left_on='model_id', right_on='model_bad_id', suffixes=('_elo', '_df'), how='inner')
elos_named.drop(['model_bad_id'], axis=1, inplace=True)
elos_named.rename(columns={"model_bad_id": "name", "model_bad_step": "step"}, inplace=True)
elos_named

,model_id,elo,model_bad_name,step
0,0,895.745899,P2_PPO_universal_h=32_more_samples__1__1686135602,12800000
1,1,1075.477812,P2_PPO_universal_h=32_more_samples__1__1686135602,25600000
2,2,1214.210467,P2_PPO_universal_h=32_more_samples__1__1686135602,38400000
3,3,1293.816067,P2_PPO_universal_h=32_more_samples__1__1686135602,51200000
4,4,1346.646065,P2_PPO_universal_h=32_more_samples__1__1686135602,64000000
...,...,...,...,...
122,122,982.032018,PPO_universal_h=32__1__1685389243,64000000
123,123,1036.673659,PPO_universal_h=32__1__1685389243,76800000
124,124,1054.828995,PPO_universal_h=32__1__1685389243,80000000
125,random,724.257450,random,0


In [6]:
elos_named.sort_values(by=['elo'], ascending=False)


,model_id,elo,model_bad_name,step
9,9,1386.212605,P2_PPO_universal_h=32__1__1686135602,64000000
11,11,1385.543107,P2_PPO_universal_h=32__1__1686135602,80000000
10,10,1372.873481,P2_PPO_universal_h=32__1__1686135602,76800000
38,38,1364.368534,P2_PPO_universal_RNN_h=64_rnn=128__1__1685970607,64000000
4,4,1346.646065,P2_PPO_universal_h=32_more_samples__1__1686135602,64000000
...,...,...,...,...
83,83,631.738218,T_PPO_universal_RNN_h=64_rnn=128__1__1685745084,12800000
48,48,631.618624,P2_PPO_universal_RNN_h=32_rnn=128__1__1685970585,12800000
69,69,588.037300,T_PPO_universal_RNN_h=32_rnn=128__1__1685745112,12800000
90,90,567.179048,T_PPO_universal_RNN_h=64_rnn=64__1__1685745066,12800000
